### Importing all the libraries

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Using CUDA if it is available

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Model Parameters

In [14]:
input_size = 96
sequence_length = 32
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.0001
batch_size = 64
num_epochs = 2

### Creating a Recurrent Neural Network Model

In [15]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Forward Propagation
        out, _ = self.rnn(x, h0)
        out = out.reshape(x.shape[0],-1)
        out = self.fc(out)
        return out

### Checking the model on random data

In [16]:
model = RNN(96,256,2,10)
x = torch.randn(64,3,32,32)
y = torch.zeros(64,1,32,96)
for i in range(x.shape[0]):
    temp = torch.cat([x[i][j] for j in range(x.shape[1])], dim=1)
    y[i] = temp

In [17]:
y = y.squeeze(1)
print(model(y).shape)

torch.Size([64, 10])


In [18]:
arr = model(y)

In [19]:
arr

tensor([[-9.1175e-02, -1.0364e-01,  5.0688e-03,  3.8312e-03, -8.6239e-02,
         -4.6859e-02,  1.2940e-01, -2.2551e-02, -1.5549e-01,  2.3743e-02],
        [ 9.7047e-03,  5.7825e-02, -3.2961e-02, -6.9764e-02,  3.3106e-01,
          2.3889e-01,  2.0269e-01, -2.2587e-02, -6.3375e-02, -2.4675e-01],
        [-5.2968e-02,  9.2787e-02, -1.3837e-02,  6.2412e-02,  6.8581e-02,
         -8.9278e-02, -1.8089e-01, -5.5880e-02, -3.1822e-01, -1.7971e-01],
        [-1.6568e-02, -1.2827e-01, -1.9070e-01, -2.9958e-02,  5.9358e-02,
          9.0504e-03, -3.7893e-01, -1.8232e-01, -1.6324e-02, -2.1260e-01],
        [ 4.2774e-02,  2.0241e-01, -2.2559e-01,  1.0465e-01, -1.1867e-01,
         -6.6149e-02,  3.8775e-02, -2.6456e-02,  9.9592e-02, -6.8644e-02],
        [ 2.1357e-01, -4.8056e-02,  1.4445e-01, -1.2700e-01, -3.1983e-02,
         -6.6256e-02,  8.2118e-02, -1.5729e-01,  4.9228e-02,  6.7447e-02],
        [ 1.8715e-01,  4.0879e-03, -2.1306e-01, -1.3676e-01, -8.9645e-03,
          1.8195e-01, -2.8554e-0

In [20]:
torch.einsum("ij->i",arr)

tensor([-0.3439,  0.4047, -0.6670, -1.0873, -0.0173,  0.1262, -0.3359, -0.1565,
        -0.1374, -0.0994,  0.1067,  0.1923, -0.6997, -0.6713, -0.4238,  0.3540,
        -0.4442,  0.1931, -0.3580, -0.0869, -0.3072,  0.4597, -0.4839, -0.4864,
         0.4218, -0.0877, -0.1812,  0.7049, -0.2677, -0.5266, -0.6704, -0.2134,
        -0.5001,  0.1388, -0.3804, -0.4037, -0.2192,  0.0983, -1.1133,  0.2691,
         0.1546,  0.4222,  0.2366, -0.1672, -0.2679,  0.0122,  0.2089,  0.3730,
        -0.5901,  0.2103,  0.0779, -0.1232, -0.5638, -0.2399,  0.4340,  0.7859,
        -0.3657,  0.0859, -0.0296, -0.9558, -0.4571,  0.5460,  0.0381, -0.0733],
       grad_fn=<SumBackward1>)

### Data Loading

In [21]:
train_dataset = datasets.CIFAR10(root='cifar_data/',train=True, transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.CIFAR10(root='cifar_data/',train=False, transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### Intialize the Model

In [22]:
model = RNN(input_size,hidden_size,num_layers,num_classes).to(device)

### Loss and Optimizer

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [24]:
for epoch in range(num_epochs):
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get the data to CUDA if possible
        #data = data.to(device=device).view(data.shape[0],1,32,96).squeeze(1)
        shaped_data = torch.zeros(data.shape[0],1,32,96).to(device=device)
        for i in range(data.shape[0]):
            temp = torch.cat([data[i][j] for j in range(data.shape[1])], dim=1)
            shaped_data[i] = temp
        shaped_data = shaped_data.squeeze(1)
        targets = targets.to(device=device)
        
        # forward
        scores = model(shaped_data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Descent
        optimizer.step()

### Checking the accuracy of the Model

In [25]:
def check_accuracy(loader, model):
    
    if loader.dataset.train:
        print("Checking accuracy on training data")
        
    else :
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    #model.eval()
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).view(x.shape[0],1,32,96).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
            for i in range(y.size(0)):
                label = y[i]
                pred = predictions[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
            
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    #model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
Accuracy of plane: 50.48 %
Accuracy of car: 50.2 %
Accuracy of bird: 9.04 %
Accuracy of cat: 4.6 %
Accuracy of deer: 5.76 %
Accuracy of dog: 13.98 %
Accuracy of frog: 0.1 %
Accuracy of horse: 0.86 %
Accuracy of ship: 3.58 %
Accuracy of truck: 2.38 %
Got 7049/50000 with accuracy 14.10
Checking accuracy on test data
Accuracy of plane: 52.0 %
Accuracy of car: 48.7 %
Accuracy of bird: 9.7 %
Accuracy of cat: 4.4 %
Accuracy of deer: 6.5 %
Accuracy of dog: 13.7 %
Accuracy of frog: 0.0 %
Accuracy of horse: 0.7 %
Accuracy of ship: 3.8 %
Accuracy of truck: 2.2 %
Got 1417/10000 with accuracy 14.17
